# "European Covid data exploration"
> "Exploring which countries have had the highest and lowest covid numbers in Europe"

- toc: true
- branch: master
- badges: true
- comments: false
- author: Ifan Johnston
- categories: [covid]

# Importing and preparing the data

We will be looking at data from the following countries:

- Italy
- Austria
- Germany
- Belgium
- France
- United Kingdom
- Portugal

We begin by importing the data, and adding some new features so that we can compare the data from different countries. For example we calculate 'confirmed cases per 100k population', 'deaths per 100k' and 'new cases' since these are not initially in the dataset. This data is collected and preprocessed in [this file](https://github.com/idjotherwise/nlp-otherwise/blob/master/src/update_datasets.py).

In [1]:
#collapse

import altair as alt
import pandas as pd

x_small_url = "https://raw.githubusercontent.com/idjotherwise/nlp-otherwise/master/data_sets/european_covid.csv"

Here is a random sample of 5 rows from the dataset.

In [2]:
x_small = pd.read_csv(x_small_url)
x_small.sample(5)

,id,date,vaccines,confirmed,tests,recovered,deaths,population,confirmed_per,deaths_per,ratio,tests_per,vaccines_per,new_cases,new_cases_per
4014,France,2021-01-08,93468.0,2742349.0,37952950.0,177157.0,NaN,66977107.0,4094.457230,NaN,NaN,56665.555889,0.001396,19621.0,29.295084
275,Italy,2020-11-25,NaN,1480874.0,20956187.0,637149.0,52028.0,60421760.0,2450.895174,86.108051,3.513331,34683.178709,NaN,25852.0,42.785910
1572,Belgium,2021-10-24,17471811.0,1321910.0,21339973.0,NaN,26048.0,11433256.0,11561.973247,227.826614,1.970482,186648.256630,1.528157,2450.0,21.428716
1210,Belgium,2020-10-27,NaN,396693.0,4749745.0,23035.0,11355.0,11433256.0,3469.641544,99.315541,2.862415,41543.240176,NaN,22191.0,194.091692
1216,Belgium,2020-11-02,NaN,467951.0,5073519.0,25999.0,12379.0,11433256.0,4092.893573,108.271869,2.645362,44375.101896,NaN,13843.0,121.076621


# Plotting the data

We will first look at the total numbers of cases and deaths in each country, before moving on to cases and deaths per 100k population.

{% include info.html text="In each of the charts below, you can click on the legend to filter the lines shown" %}

## Total cases per 100,000

In [3]:
#collapse

leg_selection = alt.selection_multi(fields=['id'], bind='legend')

alt.Chart(x_small_url).mark_line().encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("confirmed_per:Q", axis=alt.Axis(title='Confirmed per 100k')),
    tooltip=['id:N', 'confirmed_per:Q'],
    color=alt.Color('id:N', legend=alt.Legend(title="Countries")),
    opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).transform_filter(alt.datum.confirmed_per>0).add_selection(leg_selection).properties(title='Total number of cases per 100,000 population for selected European Countries', width=600).interactive()


alt.Chart(...)

## Total deaths per 100,000

In [4]:
#collapse
alt.Chart(x_small_url).mark_line().encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("deaths_per:Q", axis=alt.Axis(title='Deaths per 100k'), impute=alt.ImputeParams(value=50)),
    tooltip=["id:N", "deaths_per:Q", "yearmonthdate(date):T"],
    color=alt.Color('id:N', legend=alt.Legend(title="Countries")),
    opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).transform_filter(alt.datum.deaths_per>0).add_selection(leg_selection).properties(title='Number of deaths per 100,000 population for selected European Countries', width=600).interactive()

alt.Chart(...)

## Two week incidence rate

In [5]:
#collapse
brush = alt.selection(type='interval', encodings=['x'])

base = alt.Chart(x_small_url).mark_line().transform_filter(alt.datum.new_cases_per>0).transform_window(
    rolling_mean='sum(new_cases_per)',
    frame=[-14, 0],
    groupby=['id:N']
).encode(
    x=alt.X("yearmonthdate(date):T",
            axis=alt.Axis(title='Date')
           ),
    y=alt.Y("rolling_mean:Q",
            axis=alt.Axis(title='Incidence rate')
           ),
    tooltip=['id:N', 'rolling_mean:Q'],
    color=alt.Color('id:N', legend=alt.Legend(title="Countries")),
    opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(
    width=600,
    height=400,
    title='Number of new cases per 100,000 over two weeks for selected countries'
)

upper = base.encode(
    alt.X('yearmonthdate(date):T',axis=alt.Axis(title='Date'),
          scale=alt.Scale(domain=brush))
)

lower = base.properties(
    height=60
).add_selection(brush)

upper & lower

alt.VConcatChart(...)

The ratio of confirmed cases and deaths gives an indication of what the case fatality rate is - it seems to be between 2 and 3%, assuming that the countries listed here are catching all positive cases (which they probably aren't, so it's likely lower than this).

## Case fatality rate

In [6]:
#collapse
base = alt.Chart(x_small_url).mark_line().transform_filter(alt.datum.ratio>0).encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("ratio:Q", axis=alt.Axis(title='Ratio of deaths per case')),
    tooltip='id:N',
    color=alt.Color('id:N', legend=alt.Legend(title="Countries")),
opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(title='The ratio of deaths to confirmed cases (case fatality rate)', width=600)

upper = base.encode(
    alt.X('yearmonthdate(date):T',axis=alt.Axis(title='Date'),
          scale=alt.Scale(domain=brush))
)

lower = base.properties(
    height=60
).add_selection(brush)

upper & lower

alt.VConcatChart(...)

# Vaccines

In the chart below we plot the number vaccines given per population - this means that if the number is 1, then the country has given the equivalent of 1 shot for each person in the country. Since not everyone in the countries are eligible to get the vaccine, a ratio of 1 means that many people have recieved two jabs. Note also that some kinds of vaccines (the J&J's [Janssen vaccine](https://www.cdc.gov/coronavirus/2019-ncov/vaccines/different-vaccines/janssen.html), for example) only require 1 shot so the goal is not neccesarily to reach exactly 2 shots per person in the whole country.

In [7]:
#collapse
alt.Chart(x_small_url).mark_line().transform_filter(alt.datum.vaccines_per>0).encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("vaccines_per:Q", axis=alt.Axis(title='Number of vaccines given')),
    tooltip=['id:N', 'vaccines_per:Q'],
    color=alt.Color('id:N', legend=alt.Legend(title="Countries")),
    opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(title='Number of vaccines given', width=600).interactive()

alt.Chart(...)